# Public Training with Differential Privacy

By training your model with differential privacy, you can ensure that the model is not memorizing sensitive informations about the training set. If the model is not trained with differential privacy, attackers could reveal some private imformations by just querring the deployed model. Two common attacks are [membership inference](https://www.cs.cornell.edu/~shmat/shmat_oak17.pdf) and [model inversion](https://www.cs.cmu.edu/~mfredrik/papers/fjr2015ccs.pdf).

To learn mode about [TensorFlow Privacy](https://github.com/tensorflow/privacy) you can read this [excellent blog post](http://www.cleverhans.io/privacy/2019/03/26/machine-learning-with-differential-privacy-in-tensorflow.html). Before running this notebook, please install [TensorFlow Privacy](https://github.com/tensorflow/privacy)

In [2]:
"""Training a CNN on MNIST with Keras and the DP SGD optimizer.
source: https://github.com/tensorflow/privacy/blob/master/tutorials/mnist_dpsgd_tutorial_keras.py
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from privacy.analysis.rdp_accountant import compute_rdp
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

In [3]:
dpsgd = True            # If True, train with DP-SGD
learning_rate = 0.15    # Learning rate for training
noise_multiplier = 1.1  # Ratio of the standard deviation to the clipping norm
l2_norm_clip = 1.0      # Clipping norm
batch_size = 250        # Batch size
epochs = 20             # Number of epochs
microbatches = 50       # Number of microbatches


def compute_epsilon(steps):
    """Computes epsilon value for given hyperparameters."""
    if noise_multiplier == 0.0:
        return float('inf')
    orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
    sampling_probability = batch_size / 60000
    rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=steps,
                    orders=orders)
    # Delta is set to 1e-5 because MNIST has 60000 training points.
    return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]


def load_mnist():
    """Loads MNIST and preprocesses to combine training and validation data."""
    train, test = tf.keras.datasets.mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
    test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)
    
    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

    assert train_data.min() == 0.
    assert train_data.max() == 1.
    assert test_data.min() == 0.
    assert test_data.max() == 1.

    return train_data, train_labels, test_data, test_labels

tf.logging.set_verbosity(tf.logging.INFO)
if dpsgd and batch_size % microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

# Load training and test data.
train_data, train_labels, test_data, test_labels = load_mnist()

model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                             strides=2,
                             padding='same',
                             activation='relu',
                             input_shape=(28, 28, 1)),
      tf.keras.layers.AveragePooling2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                             strides=2,
                             padding='valid',
                             activation='relu'),
      tf.keras.layers.AveragePooling2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

if dpsgd:
    optimizer = DPGradientDescentGaussianOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=microbatches,
        learning_rate=learning_rate)
    # Compute vector of per-example loss rather than its mean over a minibatch.
    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)
else:
    optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Compile model with Keras
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train model with Keras
model.fit(train_data, train_labels,
        epochs=epochs,
        validation_data=(test_data, test_labels),
        batch_size=batch_size)

# Compute the privacy budget expended.
if dpsgd:
    eps = compute_epsilon(epochs * 60000 // batch_size)
    print('For delta=1e-5, the current epsilon is: %.2f' % eps)
else:
    print('Trained with vanilla non-private SGD optimizer')

W0812 16:43:41.723454 4616672704 deprecation.py:506] From /anaconda3/envs/pysyft_36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0812 16:43:42.004826 4616672704 deprecation.py:323] From /anaconda3/envs/pysyft_36/lib/python3.6/site-packages/tensorflow/python/ops/clip_ops.py:286: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0812 16:43:42.030165 4616672704 deprecation_wrapper.py:119] From /Users/yanndupis/Documents/dropoutlabs/privacy/privacy/dp_query/gaussian_query.py:101: The name tf.random_normal is deprecated. Please use tf.random.n

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 163s 3ms/sample - loss: 1.3740 - acc: 0.5710 - val_loss: 0.7501 - val_acc: 0.7694
Epoch 2/20
60000/60000 [==============================] - 173s 3ms/sample - loss: 0.6540 - acc: 0.8002 - val_loss: 0.5825 - val_acc: 0.8213
Epoch 3/20
60000/60000 [==============================] - 212s 4ms/sample - loss: 0.5638 - acc: 0.8326 - val_loss: 0.5217 - val_acc: 0.8486
Epoch 4/20
60000/60000 [==============================] - 226s 4ms/sample - loss: 0.4947 - acc: 0.8555 - val_loss: 0.4364 - val_acc: 0.8750
Epoch 5/20
60000/60000 [==============================] - 223s 4ms/sample - loss: 0.4756 - acc: 0.8684 - val_loss: 0.4666 - val_acc: 0.8834
Epoch 6/20
60000/60000 [==============================] - 196s 3ms/sample - loss: 0.4429 - acc: 0.8806 - val_loss: 0.3911 - val_acc: 0.8983
Epoch 7/20
60000/60000 [==============================] - 181s 3ms/sample - loss: 0.3975 - acc: 0.8966 - val_l

In [ ]:
model.save('short-dnn.h5')